In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [6]:
tf.test.is_built_with_cuda()

True

In [7]:
!python --version

Python 3.7.16


In [ ]:

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        if agent_id == "pursuer_0":
            return "pursuer_0"
        if agent_id == "pursuer_1":
            return "pursuer_1"
        if agent_id == "pursuer_2":
            return "pursuer_2"
    
    multi_agent_config = DQNConfig()
    multi_agent_config = multi_agent_config.environment("meta_environment", env_config={'env':env, 'action_sample_n':action_sample_n, 'bonus_reward':False})
    multi_agent_config = multi_agent_config.multi_agent(
                                policies={
                            "policy0": (None, meta_environment.observation_space, meta_environment.action_space, {}),
                            "policy1": (None, meta_environment.observation_space, meta_environment.action_space, {}),
                            "policy2": (None, meta_environment.observation_space, meta_environment.action_space, {}),},
                                policy_mapping_fn=policy_mapping_fn)
multi_agent_config = multi_agent_config.framework("tf2")



In [10]:
import argparse

import ray

from ray import air, tune
from ray.tune.registry import register_env
from ray.rllib.algorithms.apex_ddpg import ApexDDPGConfig
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from pettingzoo.sisl import waterworld_v4

In [11]:
from ray import air, tune
from ray.tune.registry import register_env
from ray.rllib.algorithms.apex_ddpg import ApexDDPGConfig


In [47]:
args = {
    "num_gpus": len(tf.config.experimental.list_physical_devices('GPU')),
    "as_test": False
}
def env_creator(args):
    
    return PettingZooEnv(waterworld_v4.env(n_pursuers=3, n_evaders=3, n_poisons=5, n_coop=2))

env = env_creator({})
register_env("waterworld", env_creator)

    

In [13]:
config = (
        ApexDDPGConfig()
        .environment("waterworld")
        .resources(num_gpus=args['num_gpus'])
        .rollouts(num_rollout_workers=2)
        .multi_agent(
            policies=env.get_agent_ids(),
            policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
        )
    )

if args['as_test']:
    
        # Only a compilation test of running waterworld / independent learning.
    stop = {"training_iteration": 1}
else:
    stop = {"episodes_total": 10}

In [19]:


    tune.Tuner(
        "APEX_DDPG",
        run_config=air.RunConfig(
            stop=stop,
            checkpoint_config=air.CheckpointConfig(
                checkpoint_frequency=100,
            ),
        ),
        param_space=config
#         resources_per_trial={"cpu": num_cpu_cores, "gpu": args['num_gpus']}
    ).fit()

(ApexDDPG pid=12144) 2023-05-04 11:31:38,457	WARNING algorithm_config.py:636 -- Cannot create ApexDDPGConfig from given `config_dict`! Property __stdout_file__ not supported.
(ApexDDPG pid=12144) 2023-05-04 11:31:38,752	INFO algorithm.py:528 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=15912) 2023-05-04 11:31:49,528	WARNING env.py:286 -- Your MultiAgentEnv <PettingZooEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-05-04 11:31:49,622	ERROR trial_runner.py:1450 -- Trial APEX_DDPG_waterworld_bb67e_00000: Error happened when processing _ExecutorEventType.TRAINING_RESULT.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\tune\execution\ray_trial_exe

Trial name,trial_id
APEX_DDPG_waterworld_bb67e_00000,bb67e_00000


2023-05-04 11:31:49,639	ERROR tune.py:941 -- Trials did not complete: [APEX_DDPG_waterworld_bb67e_00000]
2023-05-04 11:31:49,639	INFO tune.py:946 -- Total run time: 23.77 seconds (23.77 seconds for the tuning loop).


ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': 'bb67e_00000'},
    path='C:\\Users\\acqua\\ray_results\\APEX_DDPG\\APEX_DDPG_waterworld_bb67e_00000_0_2023-05-04_11-31-25',
    checkpoint=None
  )
]>

(ApexDDPG pid=12144) 2023-05-04 11:31:49,622	ERROR worker.py:844 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::ApexDDPG.__init__() (pid=12144, ip=127.0.0.1, repr=ApexDDPG)
(ApexDDPG pid=12144)   File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
(ApexDDPG pid=12144)   File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
(ApexDDPG pid=12144)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
(ApexDDPG pid=12144)   File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
(ApexDDPG pid=12144)     return method(__ray_actor, *args, **kwargs)
(ApexDDPG pid=12144)   File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(ApexDDPG pid=12144)     return method(self, *_args, **_kwargs)
(ApexDDPG pid=12144)   File "C:\Use

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

In [5]:
config = (
        PPOConfig()
        .environment("waterworld")
        .resources(num_gpus=args['num_gpus'])
        .rollouts(num_rollout_workers=2)
        .multi_agent(
            policies=env.get_agent_ids(),
            policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
        )
    )

if args['as_test']:
    
        # Only a compilation test of running waterworld / independent learning.
    stop = {"training_iteration": 1}
else:
    stop = {"episodes_total": 10000}
    

NameError: name 'args' is not defined

In [15]:


    tune.Tuner(
        "PPO",
        run_config=air.RunConfig(
            stop=stop,
            checkpoint_config=air.CheckpointConfig(
                checkpoint_frequency=100,
            ),
        ),
        param_space=config
#         resources_per_trial={"cpu": num_cpu_cores, "gpu": args['num_gpus']}
    ).fit()

2023-05-04 11:50:23,925	INFO worker.py:1622 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-05-04 11:50:25,677	INFO tune.py:219 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


(PPO pid=27380) 2023-05-04 11:50:36,379	WARNING algorithm_config.py:636 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=27380) 2023-05-04 11:50:36,659	INFO algorithm.py:528 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-05-04 11:50:47,711	ERROR trial_runner.py:1450 -- Trial PPO_waterworld_62c97_00000: Error happened when processing _ExecutorEventType.TRAINING_RESULT.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\tune\execution\ray_trial_executor.py", line 1231, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\wo

Trial name,trial_id
PPO_waterworld_62c97_00000,62c97_00000


2023-05-04 11:50:47,727	ERROR ray_trial_executor.py:884 -- An exception occurred when trying to stop the Ray actor:Traceback (most recent call last):
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\tune\execution\ray_trial_executor.py", line 874, in _resolve_stop_event
    ray.get(future, timeout=timeout)
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\worker.py", line 2523, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=27380, ip=127.0.0.1, repr=PPO)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "C:\Us

ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': '62c97_00000'},
    path='C:\\Users\\acqua\\ray_results\\PPO\\PPO_waterworld_62c97_00000_0_2023-05-04_11-50-25',
    checkpoint=None
  )
]>

(PPO pid=27380) 2023-05-04 11:50:47,700	ERROR worker.py:844 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=27380, ip=127.0.0.1, repr=PPO)
(PPO pid=27380)   File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
(PPO pid=27380)   File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
(PPO pid=27380)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
(PPO pid=27380)   File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
(PPO pid=27380)     return method(__ray_actor, *args, **kwargs)
(PPO pid=27380)   File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(PPO pid=27380)     return method(self, *_args, **_kwargs)
(PPO pid=27380)   File "C:\Users\acqua\anaconda3\envs\CMSC_742\lib\site-packages\ray\

In [ ]:
# Defining Custom Model with modified Loss
from ray.rllib.models import ModelCatalog


import numpy as np

from ray.rllib.models.modelv2 import ModelV2, restore_original_dimensions
from ray.rllib.models.tf.tf_action_dist import Categorical
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.offline import JsonReader

tf1, tf, tfv = try_import_tf()
torch, nn = try_import_torch()


class OIRLossModel(TFModelV2):
    """Custom model that adds an imitation loss on top of the policy loss."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        super().__init__(obs_space, action_space, num_outputs, model_config, name)

        self.fcnet = FullyConnectedNetwork(
            self.obs_space, self.action_space, num_outputs, model_config, name="fcnet"
        )

    @override(ModelV2)
    def forward(self, input_dict, state, seq_lens):
        # Delegate to our FCNet.
        return self.fcnet(input_dict, state, seq_lens)

    @override(ModelV2)
    def value_function(self):
        # Delegate to our FCNet.
        return self.fcnet.value_function()

    @override(ModelV2)
    def custom_loss(self, policy_loss, loss_inputs):
        # Create a new input reader per worker.
        reader = JsonReader(self.model_config["custom_model_config"]["input_files"])
        input_ops = reader.tf_input_ops()

        # Define a secondary loss by building a graph copy with weight sharing.
        obs = restore_original_dimensions(
            tf.cast(input_ops["obs"], tf.float32), self.obs_space
        )
        logits, _ = self.forward({"obs": obs}, [], None)

        # You can also add self-supervised losses easily by referencing tensors
        # created during _build_layers_v2(). For example, an autoencoder-style
        # loss can be added as follows:
        # ae_loss = squared_diff(
        #     loss_inputs["obs"], Decoder(self.fcnet.last_layer))
        print("FYI: You can also use these tensors: {}, ".format(loss_inputs))

        # Compute the IL loss.
        action_dist = Categorical(logits, self.model_config)
        self.policy_loss = policy_loss
        self.imitation_loss = tf.reduce_mean(-action_dist.logp(input_ops["actions"]))
        return policy_loss + 10 * self.imitation_loss

    def metrics(self):
        return {
            "policy_loss": self.policy_loss,
            "imitation_loss": self.imitation_loss,
        }


ModelCatalog.register_custom_model("OIR_loss", OIRLossModel)

In [74]:
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.algorithms.ppo import PPOConfig

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        if agent_id == "pursuer_0":
            return "pursuer_0"
        if agent_id == "pursuer_1":
            return "pursuer_1"
        if agent_id == "pursuer_2":
            return "pursuer_2"



config = (
        PPOConfig()
        .environment("waterworld")
        .resources(num_gpus=0)     # args['num_gpus']
        .rollouts(num_rollout_workers=2)
        .multi_agent(
            policies=env.get_agent_ids(),
            policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
        )
        .multi_agent(
                    policies={
                "pursuer_0": (None, env.observation_space, env.action_space, {}),
                "pursuer_1": (None, env.observation_space, env.action_space, {}),
                "pursuer_2": (None, env.observation_space, env.action_space, {}),},
                    policy_mapping_fn=policy_mapping_fn)
    
#         .training(
#             model={
#                 "custom_model": "OIR_loss",
#                 "custom_model_config": {
#                     "ip1": ,
#                 },
#             },
#             _enable_learner_api=False,
#         )
    )

algo = config.build()
# CustomPolicy = PPOTFPolicy.with_updates(
#     name="MyCustomPPOTFPolicy",
#     loss_fn=some_custom_loss_fn)

# CustomTrainer = PPOTrainer.with_updates(
#     default_policy=CustomPolicy)

(RolloutWorker pid=24384) 2023-05-14 22:05:41,513	WARNING env.py:286 -- Your MultiAgentEnv <PettingZooEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-05-14 22:05:41,604	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [78]:
env.reset()

({'pursuer_0': array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.

In [126]:
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO, PPOConfig
import numpy as np
from sklearn.neighbors import KernelDensity

for i in range(1):
    result = algo.train()


    # Generate some data by running the policy in the environment
    n_samples = 1000
    states = []
    actions = []
    init_states = []
    state , _ = env.reset()
    for t in range(n_samples):
#         action = algo.get_policy('pursuer_0').compute_actions(state)
        action = {'pursuer_0': np.array([0.1,0.1]), 'pursuer_1': np.array([0.1,0.1]), 'pursuer_2': np.array([0.1,0.1])}
        for _ in range(3):
            next_state, reward, term, trunc , _ = env.step(action)
            states.append(state.values())
            actions.append(action)
        if done:
            state, _ = env.reset()
        else:
            state = next_state

    # Fit a kernel density estimator to the state-action pairs
    bandwidth = 0.1
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
    data = np.concatenate([states, actions], axis=1)
    kde.fit(data)

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

...

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
[WARNING]: Received an action [0.1 0.1] that was outside actio

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
next_state, reward, done, _ , _ = env.step({'pursuer_0': np.array([0.1,0.1]), 'pursuer_1': np.array([0.1,0.1]), 'pursuer_2': np.array([0.1,0.1])})
next_state.values()

In [128]:
state, _ = env.reset()
a = 
a[-1]

TypeError: 'dict_values' object is not subscriptable

In [ ]:
obs[0]['pursuer_0']

In [ ]:
algo.get_policy('pursuer_0').compute_single_action(obs[0]['pursuer_0'])

In [32]:
env.reset()[0].keys()

dict_keys(['pursuer_0'])

In [34]:
!pip install -U scikit-learn


     ---------------------------------------- 7.1/7.1 MB 22.8 MB/s eta 0:00:00
     ------------------------------------- 298.0/298.0 kB 18.0 MB/s eta 0:00:00


In [44]:
dir(algo)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allow_unknown_configs',
 '_allow_unknown_subkeys',
 '_annotated',
 '_automatic_evaluation_duration_fn',
 '_before_evaluate',
 '_checkpoint_cls',
 '_checkpoint_info_to_algorithm_state',
 '_close_logfiles',
 '_compile_iteration_results',
 '_counters',
 '_create_checkpoint_dir',
 '_create_local_replay_buffer_if_necessary',
 '_create_logger',
 '_env_id',
 '_episode_history',
 '_episodes_to_be_collected',
 '_episodes_total',
 '_evaluate_async',
 '_export_model',
 '_get_env_id_and_creator',
 '_get_latest_available_checkpoint',
 '_get_latest_local_available_checkpoint',
 '_

In [87]:
env.action_space

Box(-1.0, 1.0, (2,), float32)

In [56]:
env.observation_space

Box(-1.4142135, 1.4142135, (242,), float32)

In [80]:
from pettingzoo.sisl import waterworld_v4

In [82]:
a = PettingZooEnv(waterworld_v4.env())

In [83]:
a.reset()

({'pursuer_0': array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 0.03292426, 0.12405557,
         0.20976506, 0.2863068 , 0.35033557, 0.39905298, 0.4303299 ,
         0.44279927, 0.43591624, 0.40998155, 0.3661287 , 0.3062743 ,
         0.23303422, 0.14960943, 0.05964599, 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 0.92640644, 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 